In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
folder_path = "C:/Users/qhrrl/CodeExamples/3.Paper/0.Dataset"

In [3]:
pd_cur_train = pd.read_csv(folder_path + '/cur_training_all.csv')
pd_cur_val = pd.read_csv(folder_path + '/cur_validation_all.csv')

In [6]:
print(len(pd_cur_train))
print(len(pd_cur_val))

305534
38210


In [7]:
pd_cur_train.head(3)

,Date,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,...,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x,WATT,LABEL
0,2020-11-13 07:43:41,3.643555,3.883789,4.124023,2.152260,2.154582,2.290735,7.126953,7.527344,7.687500,...,3.553066,3.763136,0.021705,0.023957,0.020911,0.057528,0.057195,0.057517,2.2,0
1,2020-11-13 07:43:41,3.643555,3.803711,4.164062,2.161249,2.163811,2.301946,7.126953,7.447266,7.727539,...,3.533756,3.747315,0.009876,0.010175,0.010687,0.011616,0.013211,0.010941,2.2,0
2,2020-11-13 07:43:41,3.683594,3.803711,4.164062,2.152920,2.151939,2.289333,7.166992,7.367188,7.727539,...,3.519853,3.728701,0.017622,0.011568,0.014016,0.012954,0.016530,0.011328,2.2,0


In [10]:
np_cur_train = pd_cur_train.to_numpy()
np_cur_val = pd_cur_val.to_numpy()

# Extract Labels

In [11]:
cur_train_labels = np_cur_train[:,-1]
cur_val_labels = np_cur_val[:, -1]
print(cur_train_labels)
print(cur_val_labels)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


In [12]:
pd_cur_train = pd_cur_train.drop(columns = ['Date','WATT','LABEL'])
pd_cur_val = pd_cur_val.drop(columns = ['Date','WATT','LABEL'])

# 딥러닝 용 데이터로 전처리

In [13]:
pd_cur_train.head(3)

,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,R_RMS,...,T_1x,R_2x,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x
0,3.643555,3.883789,4.124023,2.152260,2.154582,2.290735,7.126953,7.527344,7.687500,2.393055,...,0.003780,3.536451,3.553066,3.763136,0.021705,0.023957,0.020911,0.057528,0.057195,0.057517
1,3.643555,3.803711,4.164062,2.161249,2.163811,2.301946,7.126953,7.447266,7.727539,2.405001,...,0.012369,3.521399,3.533756,3.747315,0.009876,0.010175,0.010687,0.011616,0.013211,0.010941
2,3.683594,3.803711,4.164062,2.152920,2.151939,2.289333,7.166992,7.367188,7.727539,2.394561,...,0.012448,3.506221,3.519853,3.728701,0.017622,0.011568,0.014016,0.012954,0.016530,0.011328


In [14]:
cur_train = pd_cur_train.to_numpy()
cur_val = pd_cur_val.to_numpy()

# 1. Current

## 1.1 Data

### 1.1 split data set 

In [15]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(cur_train, cur_train_labels, test_size=0.2, random_state=42)

### 1.2. Validation

In [16]:
print(x_train.shape)
print(y_train.shape)
print("="*50)
print(x_test.shape)
print(y_test.shape)
print("="*50)
x_valid = cur_val
y_valid = cur_val_labels

print(x_valid.shape)
print(y_valid.shape)

(244427, 45)
(244427,)
(61107, 45)
(61107,)
(38210, 45)
(38210,)


### 1.3 normalize

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Standard Scaler를 적용하고 싶을 경우 아래 Code를 실행

stder = StandardScaler()
stder.fit(x_train)

x_train = stder.transform(x_train)
x_valid = stder.transform(x_valid)
x_test = stder.transform(x_test)

# Robust Scaler를 적용하고 싶을 경우 아래 주석을 해제하고 Code를 실행

# rscaler = RobustScaler() 
# rscaler.fit(x_train)
# x_train = rscaler.transform(x_train)
# x_valid = rscaler.transform(x_valid)

주제가 피쳐에 대한 강건함이므로, 스케일러에 대한 고민도 해야할 것이다.

본 데이터는 Multivariate이며, 1개의 Channel만 갖는다. 우리는 multivariate variable with one channel 형태로 바꿔주고자 하며, 이는 아래와 같다.

In [ ]:
# CNN, RNN 모델의 입력 변수는 채널 축 1개 차원을 확장시킨 3D Tensor이다.
# 즉 아래 CNN, RNN 모델은 x_train, x_valid, x_test를 입력 변수로 넣지 않고, 
# x_train_exp, x_valid_exp, x_test_exp을 넣는다. 
x_train_exp = np.expand_dims(x_train, -1)  # 채널 축 1개 차원을 확장 시킨(Expand) X_train
x_valid_exp = np.expand_dims(x_valid, -1)  # 채널 축 1개 차원을 확장 시킨(Expand) X_vaild 
x_test_exp = np.expand_dims(x_test, -1)  # 채널 축 1개 차원을 확장 시킨(Expand) X_test

# 위와 동일한 코드
# x_train_exp = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
# x_valid_exp = x_train.reshape((x_valid.shape[0], x_valid.shape[1], 1))
# x_test_exp = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

print("x_train_exp의 형태:", x_train_exp.shape)
print("y_train_exp의 형태:", y_train.shape)

print("=" * 50)

print("x_valid_exp의 형태:", x_valid_exp.shape)
print("y_valid_exp의 형태:", y_valid.shape)

print("=" * 50)

print("x_test_exp의 형태:", x_test_exp.shape)
print("y_test_exp의 형태:", y_test.shape)

In [ ]:
# y_train = y_train.astype('int')
# y_valid = y_valid.astype('int')
# y_test = y_test.astype('int')

y_train = tf.stack(y_train)
y_valid = tf.stack(y_valid)
y_test = tf.stack(y_test)

In [ ]:
print(y_train)
print(y_valid)
print(y_test)

In [ ]:
num_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))
num_classes

In [ ]:
x_train_exp.shape

In [5]:
y_train.shape

NameError: name 'y_train' is not defined

# 3. LSTM

In [105]:
import h5py

from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Flatten, Dense

def make_lstm_model():
    model = Sequential()
    model.add(LSTM(units=256, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    return model

In [106]:
lstm = make_lstm_model()

In [107]:
lstm.build(x_train_exp.shape)

In [108]:
lstm.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (244427, 45, 256)         264192    
_________________________________________________________________
flatten_2 (Flatten)          (244427, 11520)           0         
_________________________________________________________________
dense_4 (Dense)              (244427, 100)             1152100   
_________________________________________________________________
dense_5 (Dense)              (244427, 5)               505       
Total params: 1,416,797
Trainable params: 1,416,797
Non-trainable params: 0
_________________________________________________________________


In [127]:
EPOCHS = 20
BATCH_SIZE = 64

In [128]:
lstm.compile(loss="sparse_categorical_crossentropy", 
                  optimizer='adam', 
                  metrics=["accuracy"]
                 )

In [129]:
# callbacks = [ModelCheckpoint(folder_path + '/lstm_best_model.h5', 
#                              monitor='val_loss',
#                              save_best_only=True),
#              ReduceLROnPlateau(
#                  monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
#                  ),
#              EarlyStopping(monitor="val_loss", patience=10, verbose=1)
#              ]

In [130]:
df = tf.stack(y_train)
y_train = tf.stack(df)

In [131]:
history = lstm.fit(
    x_train_exp,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    #callbacks=callbacks,
    validation_data=(x_valid_exp, y_valid),
    verbose=1
)

Epoch 1/20
3820/3820 [==============================] - 347s 90ms/step - loss: 0.0084 - accuracy: 0.9970 - val_loss: 0.9898 - val_accuracy: 0.9343
Epoch 2/20
3820/3820 [==============================] - 351s 92ms/step - loss: 0.0081 - accuracy: 0.9970 - val_loss: 0.9740 - val_accuracy: 0.9366
Epoch 3/20
3820/3820 [==============================] - 348s 91ms/step - loss: 0.0082 - accuracy: 0.9970 - val_loss: 1.1942 - val_accuracy: 0.9370
Epoch 4/20
 898/3820 [======>.......................] - ETA: 4:19 - loss: 0.0074 - accuracy: 0.9974

KeyboardInterrupt: 

In [ ]:
.#lstm = history
scores = lstm.evaluate(x_test_exp, y_test)

print("\n""Test accuracy", scores[1])
print("\n""Test loss", scores[0])
print("%s: %.2f%%" % (lstm.metrics_names[1], scores[1]*100))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

def draw_confusion_matrix(model, xt, yt, model_name):
    Y_pred = model.predict(xt)
    if model_name in ["cnn", "lstm"]:
        y_pred = np.argmax(Y_pred, axis=1)
    else: y_pred = Y_pred
    plt.figure(figsize=(3,3))
    cm = confusion_matrix(yt, y_pred)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['False', 'True'], rotation=45)
    plt.yticks(tick_marks, ['False', 'True'])
    thresh = cm.max()/1.2
    normalize = False
    fmt = '.2f' if normalize else 'd'
    for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j,i, format(cm[i,j], fmt), 
                 horizontalalignment="center", 
                 color="white" if cm[i,j] > thresh else "black", 
                 fontsize=12)
    plt.ylabel("Actual Label")
    plt.xlabel("Predicted Label")
    plt.tight_layout()
    plt.savefig(folder_path + '/{}_cm.png'.format(model_name), dpi=100, bbox_inches='tight')  # 그림 저장
    plt.show()
    print(classification_report(yt, y_pred))

In [ ]:
from sklearn.metrics import roc_curve, auc

def draw_roc(model,xt, yt, model_name):
    Y_pred = model.predict(xt)
    if model_name in ["cnn", "lstm"]:
        y_pred = np.argmax(Y_pred, axis=1)
    else: y_pred = Y_pred
    fpr, tpr, thr = roc_curve(yt, y_pred)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic {};'.format(model_name))
    plt.legend(loc="lower right")
    plt.ion()
    plt.tight_layout()
    plt.savefig(folder_path + '/{}_roc.png'.format(model_name), dpi=100, bbox_inches='tight')  # 그림 저장
    plt.show()

In [ ]:
def plot_loss_graph(history, pic_name):
    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history["val_loss"])
    plt.title("Training & Validation Loss")
    plt.ylabel("loss", fontsize="large")
    plt.xlabel("epoch", fontsize="large")
    plt.legend(["train", "validation"], loc="best")
    plt.tight_layout()
    plt.savefig(folder_path + '/{}.png'.format(pic_name), dpi=100, bbox_inches='tight')  # 그림 저장
    plt.show()
    plt.close()

In [ ]:
def plot_prediction_graph(history, pic_name):
    plt.figure()
    plt.plot(history.history["sparse_categorical_accuracy"])
    plt.plot(history.history["val_" + "sparse_categorical_accuracy"])
    plt.title("model " + "Prediction Accuracy")
    plt.ylabel("sparse_categorical_accuracy", fontsize="large")
    plt.xlabel("epoch", fontsize="large")
    plt.legend(["train", "validation"], loc="best")
    plt.tight_layout()
    plt.savefig(folder_path + '/{}.png'.format(pic_name), dpi=100, bbox_inches='tight')  # 그림 저장
    plt.show()
    plt.close()

In [ ]:
draw_confusion_matrix(lstm, x_test_exp, y_test, "lstm")

In [ ]:
draw_roc(lstm, x_test_exp, y_test, "lstm")

In [ ]:
plot_loss_graph(lstm, "lstm")

In [ ]:
plot_prediction_graph(lstm, "lstm")